# Tutorial Part 3

**Goal** Fit hsitograms with different functions

Clearly, the histogram you've just produced contains a Z boson. How to measure the mass of the resonance etc.?

In [ ]:
import ROOT
from ROOT import TCanvas, TF1, TH1F, TTree, TFile
%jsroot on

Open the histogram that contains the mumu invariant mass (for data).

In [ ]:
f = ROOT.TFile.Open("histo_datamuon.root","READ")
f.ls()

For developing it is convenient to use the smaller file. <br>
Note. To access the variables inside the TTree you have to use
    t1.name_of_the variable

In [ ]:
h_mumu = f.h_mumu
c1 = ROOT.TCanvas("c1","My Canvas",400,400)

h_mumu.Draw("ep")
c1.Draw()

Display in semilogarithmic scale (y-axis). You can do rightclicking, or setting the gPad with semilog y axis
Now let's fit in a very simple way

In [ ]:
h_mumu.Fit("gaus")
c1.Draw()

How the fit looks like? <br>You can read the fit parameters in the fit ouput. You may want to get them as variables in your script. <br>
    `h_mumu.Fit("gaus")` 
is actually a shortcut instead of using a `TF1`. In ROOT several functions are built in (gauss, expo, polX, etc.) If you want something more complicated you have to define your own function

In [ ]:
gaussFit = ROOT.TF1("gaussfit","gaus",81 ,101)
h_mumu.Fit("gaussfit")
c1.Draw()

In [ ]:
chi2=gaussFit.GetChisquare()
ndof=gaussFit.GetNDF()
mean=gaussFit.GetParameter(1)
width=gaussFit.GetParameter(0)
e_mean=gaussFit.GetParError(1)
e_width=gaussFit.GetParError(0)

You can print these quantities in the plot

In [ ]:
latex=ROOT.TLatex()
latex.SetNDC()
latex.SetTextSize(0.03)
latex.DrawText(0.5,0.80,"Mean=%.1f GeV"%(mean))
latex.DrawText(0.5,0.75,"Width=%.1f GeV"%(width))
latex.DrawText(0.5,0.7,"chi2/ndof=%.1f/%d = %.1f"%(chi2,ndof,chi2/ndof))
c1.Draw()

The fit is far from being perfect. In fact a resonance is not really described by a gaussian but by a Breit-Wigner distribution

![BreitWigner](images/breit_wigner.png)

You have to write that as a TF1. When writing custom functions, `x` is the name of the variable, and `[#]` are the parameters that we want
to fit. In other words, `[0]` is the first parameter, `[1]` is the second, and so on.

In [ ]:
bw_A = "2*sqrt(2)*[0]*[1]*sqrt([0]*[0]*([0]*[0]+[1]*[1]))"
bw_B = "3.14159*sqrt([0]*[0]+sqrt([0]*[0]*([0]*[0]+[1]*[1])))"
bw_C = "(x*x-[0]*[0])*(x*x-[0]*[0])+[0]*[0]*[1]*[1]"
bw = "[2]*((%s)/(%s))/(%s)"%(bw_A,bw_B,bw_C)

In [ ]:
bwFit = ROOT.TF1("bwfit",bw,50 ,150)
h_mumu.Fit("bwfit")
c1.Draw()

It doesn't really work. We have to set some initial parameters. We know approximately what is M and Gamma (mass and width of Z-boson)

In [ ]:
bwFit.SetParameter(0,90)
bwFit.SetParameter(1,3)
h_mumu.Fit("bwfit","E")
c1.Draw()

## Exercise 1
Write on the plot the fit parameters of the fit

Now use the split-panel we've learned in part 2 to plot the ratio of the fit with data

In [ ]:
c5 = ROOT.TCanvas("c5","Splitted panel",400,400)
c5.cd()
pad1 = ROOT.TPad("pad1","pad1" ,0 ,0.3 ,1 ,1)
pad1.SetLogy(True)
pad1.Draw()
pad1.cd()
h_mumu.Draw("pe")
c5.Draw()


In [ ]:
ratio = h_mumu.Clone("ratio")
pad2= ROOT.TPad("pad2","pad2",0,0.05,1,0.3)
pad2.Draw()
pad2.cd()
ratio.Divide(bwFit)
ratio.Draw("pe")
c5.Draw()

## Exercise 
Make a nicer plot